In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES']='0'

import glob
import json

import cv2
import numpy as np 
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary 

from preprocess import DrivableDataset
from models import Unet
from metric import compute_miou


device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
data_dir = "bdd100k/images/100k/"
train_images = glob.glob(data_dir + "train/*.jpg")
val_images = glob.glob(data_dir + "val/*.jpg")

trainset = DrivableDataset(train_images)
valset = DrivableDataset(val_images)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)
valloader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=4)

model = Unet()

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model.to(device)
summary(model, (3, 512, 512))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

cuda:0
Using 4 GPUs!
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           1,792
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
            Conv2d-4         [-1, 64, 512, 512]          36,928
       BatchNorm2d-5         [-1, 64, 512, 512]             128
              ReLU-6         [-1, 64, 512, 512]               0
        conv_block-7         [-1, 64, 512, 512]               0
         MaxPool2d-8         [-1, 64, 256, 256]               0
     encoder_block-9  [[-1, 64, 512, 512], [-1, 64, 256, 256]]               0
           Conv2d-10        [-1, 128, 256, 256]          73,856
      BatchNorm2d-11        [-1, 128, 256, 256]             256
             ReLU-12        [-1, 128, 256, 256]               0
           Conv2d-13        [-1, 128, 256, 256]         147,584
   

In [ ]:
num_epochs = 10
train_losses, val_losses = [], []
min_val_loss = np.Inf
for epoch in range(num_epochs):
  # training loop
    model.train()
    running_loss = 0.0
    train_iou = 0.0

    for images, labels in tqdm(trainloader, total=len(trainloader), desc='Training'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        # outputs = outputs.long()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        iou = compute_miou(outputs, labels)
        train_iou += iou

    train_losses.append(running_loss/len(trainloader))

    # validation loop
    model.eval()
    val_running_loss = 0.0
    val_iou = 0.0

    with torch.no_grad():
        for images, labels in tqdm(valloader, total=len(valloader), desc='Validation'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # outputs = outputs.long()
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            iou = compute_miou(outputs, labels)
            val_iou += iou

    val_losses.append(val_running_loss/len(valloader))

    if val_running_loss/len(valloader) < min_val_loss:
        min_val_loss = val_running_loss/len(valloader)
        checkpoint = {"model": model.state_dict(), "optimizer":optimizer.state_dict(), "epoch":epoch}
        torch.save(checkpoint, "best_model.pth")
    
    log_dict = {'train_iou': train_iou, 'val_iou': val_iou, 'epoch':epoch, 'train_loss':running_loss/len(trainloader), 'val_loss':val_running_loss/len(valloader)}
    with open("log.json", 'a+') as f:
        json.dump(log_dict, f)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(trainloader)}, Val Loss: {val_running_loss/len(valloader)}")
    print(f"Epoch {epoch+1}/{num_epochs}, Train IoU: {train_iou/len(trainloader)}, Val IoU: {val_iou/len(valloader)}")



Validation:  76%|███████▌  | 119/157 [01:57<00:33,  1.15it/s]